In [14]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

#import and clean data
nRowsRead = None
stations = pd.read_csv("stations.csv", delimiter = ',', nrows = nRowsRead, na_values = na_values=['', 'N/A', 'CA'])
nRow, nCol = stations.shape
#stations = stations.drop("Org Name", axis=1)

#change times based on zone
station_list = []
station_dict = {}
station_index = []
lat_list = []
lon_list = []
yr_mo = []
months = []
chdir = []

geolocator = Nominatim(user_agent="maps")
for i, row in stations.iterrows():
    if row['Station Name'] not in station_dict.keys():
        val = len(station_list)
        station_list.append(val)
        coords = ((str(row['Latitude'])[0:2] + "." + str(row['Latitude'])[3:]), ((str(row['Longitude'])[0:4] + "." + str(row['Longitude'])[5:])))
        if "-" in row["Address 1"]:
            add = "500 Webster St, Palo Alto, CA"
        else:
            add = row["Address 1"] + ", Palo Alto, CA"
        loc = geolocator.geocode(add)
        lat_list.append(loc.latitude)
        lon_list.append(loc.longitude)
        station_dict[row['Station Name']] = (val, row['Address 1'], row['Start Date'], (row['Year'], row['Month'], row['Day']), (loc.latitude, loc.longitude))
        #print(row['Year'], row['Month'], row['Day'])
    else:
        val = station_dict[row['Station Name']][0]
        lat_list.append(station_dict[row['Station Name']][4][0])
        lon_list.append(station_dict[row['Station Name']][4][1])
    if (row['Year'], row['Month']) not in months:
        months.append((row['Year'], row['Month']))
    station_index.append(val)
    yr_mo.append((row['Year'], row['Month']))
    chdir.append(row['Charge Duration'])
    
stations['ID'] = station_index
stations["Starting"] = pd.to_datetime(stations["Start Date"])
stations["Year Month"] = yr_mo
stations["Lat"] = lat_list
stations["Lon"] = lon_list
stations["Charge Duration"] = chdir

print(months)
print(list(stations.columns.values))
# print(station_dict)
# print(stations)

/Users/jgilman/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[(2011, 7), (2011, 8), (2011, 9), (2011, 10), (2011, 11), (2011, 12), (2012, 1), (2012, 2), (2012, 3), (2012, 4), (2012, 5), (2012, 6), (2012, 7), (2012, 8), (2012, 9), (2012, 10), (2012, 11), (2012, 12), (2013, 1), (2013, 2), (2013, 3), (2013, 4), (2013, 5), (2013, 6), (2013, 7), (2013, 8), (2013, 9), (2013, 10), (2013, 11), (2013, 12), (2014, 1), (2014, 2), (2014, 3), (2014, 4), (2014, 5), (2014, 6), (2014, 7), (2014, 8), (2014, 9), (2014, 10), (2014, 11), (2014, 12), (2015, 1), (2015, 2), (2015, 3), (2015, 4), (2015, 5), (2015, 6), (2015, 7), (2015, 8), (2015, 9), (2015, 10), (2015, 11), (2015, 12), (2016, 1), (2016, 2), (2016, 3), (2016, 4), (2016, 5), (2016, 6), (2016, 7), (2016, 8), (2016, 9), (2016, 10), (2016, 11), (2016, 12), (2017, 1), (2017, 2), (2017, 3), (2017, 4), (2017, 5), (2017, 6), (2017, 7)]
['Station Name', 'MAC Address', 'Org Name', 'Start Date', 'Start Time Zone', 'End Date', 'End Time Zone', 'Transaction Date (Pacific Time)', 'Total Duration (hh:mm:ss)', 'Chargin

In [68]:
#set up map data

location = stations.groupby("ID").first()
location = location.loc[:, ["Lat", "Lon", "Station Name"]]
#print(location)

In [98]:
# #set up map
import folium 
import imgkit
from selenium import webdriver

count = 0
colors = ["#006400", "#008000", "#2E8B57", "#32CD32", "#32CD32", "#00FF7F", "#00FF7F", "#00FF7F", "#00FF7F", "#00FF7F", 
         "#98FB98", "#33FFDA", "#33FFDA", "#33FFDA", "#33F0FF", "#33F0FF",  "#33F0FF", "#33BEFF", "#33BEFF", "#33BEFF", 
         "#33BEFF", "#33BEFF", "#338AFF", "#338AFF", "#3336FF", "#3336FF", "#3336FF"]
driver = webdriver.Chrome()

for m in months:
    palo_alto = folium.Map(location=[37.43, -122.15], zoom_start=14, tiles="CartoDB positron", zoom_control = False)
    month = stations[stations["Year Month"]==m]
    month_counts =  month.groupby("ID").count()
    month_counts = month_counts.iloc[:,[0]]
    month_counts.columns= ["Monthly Visits"]
    month_visits = month_counts.join(location)
    #print(month_visits)

    #colors so stations intro'd at same time same color - color progressively lighter
    for i, row in month_visits.iterrows():
        radius = row["Monthly Visits"]/10
        color = colors[i]
        folium.CircleMarker(location=(row["Lat"], row["Lon"]), radius=radius, color=color, fill=True).add_to(palo_alto)
    in_name = "month" + str(count) + ".html"   
    out_name = "month" + str(count) + ".png"
    path = "file:///Users/jgilman/Desktop/econ1660/Paper/" + in_name
    count += 1

    palo_alto.save(in_name)
    driver.get(path)
    driver.save_screenshot(out_name)
driver.quit()

In [18]:
palo_alto

In [103]:
#set up map
import folium 
import imgkit
from selenium import webdriver
import math

codes = pd.read_csv('zipcodes.csv', delimiter = ";", usecols =['Zip', 'Latitude', 'Longitude'])
stations_up = stations.merge(codes, how = 'left', left_on = 'Driver Postal Code', right_on = 'Zip')
# stations_up = stations_up.dropna(subset=['Driver Postal Code'])
          
zips = stations_up.groupby("Driver Postal Code").first()
zips = zips.loc[:, ["Latitude_y", "Longitude_y", "Driver Postal Code"]]
count = 0

driver = webdriver.Chrome()

for m in months:
    palo_alto = folium.Map(location=[37.43, -122.15], zoom_start=11, tiles="CartoDB dark_matter", zoom_control = False)
    month = stations_up[stations_up["Year Month"]==m]
    month_counts =  month.groupby("ID").count()
    month_counts = month_counts.iloc[:,[0]]
    month_counts.columns= ["Monthly Visits"]
    month_visits = month_counts.join(location)

    zip_counts =  month.groupby("Driver Postal Code").count()
    zip_counts = zip_counts.iloc[:,[0]]
    zip_counts.columns= ["Zip Visits"]
    zip_visits = zip_counts.join(zips)
    
    #colors so stations intro'd at same time same color - color progressively lighter
    for i, row in month_visits.iterrows():
        radius = row['Monthly Visits']/10
        #color = colors[i]
        #marker for station visits 
        folium.CircleMarker(location=(row["Lat"], row["Lon"]), radius=radius, color='#FCFCFC', fill=True).add_to(palo_alto)
    for i, row in zip_visits.iterrows():
        radius = row['Zip Visits']/10
        #marker for visits by a zipcode
        if not math.isnan(row["Latitude_y"]):
            folium.CircleMarker(location=(row["Latitude_y"], row["Longitude_y"]), radius=radius, color="#32CD32", fill=True).add_to(palo_alto)
    in_name = "month" + str(count) + ".html"   
    out_name = "month" + str(count) + ".png"
    path = "file:///Users/jgilman/Desktop/econ1660/Paper/" + in_name
    count += 1

    palo_alto.save(in_name)
    driver.get(path)
    driver.save_screenshot(out_name)
driver.quit()

In [80]:
palo_alto